In [21]:
import os

files = ["eng.testa", "eng.testb", "eng.train"]
url = "https://raw.githubusercontent.com/glample/tagger/master/dataset"
if not os.path.isdir("data"):
    os.mkdir("data")
for file in files:
    with open(f"data/{file}", 'a') as the_file:
        html = urlopen(f"{url}/{file}")
        for line in html:
            the_file.write(line.decode('UTF-8'))

In [18]:
labels = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

In [22]:
import os
import gc
import logging
from random import sample
from urllib.request import urlopen
import numpy as np
import itertools
from tqdm import tqdm, trange
import sklearn as sk
import sklearn.linear_model
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

logger = logging.getLogger(__name__)

class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words: list, labels: list = []):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
                specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode="eng.testa"):
    """Creating InputExamples out of a file"""
    file_path = os.path.join(data_dir, "{}".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features

In [23]:
dataset = "eng.train"

import torch

from torch.utils.data import TensorDataset
from transformers import BertTokenizer
# from gdpr.data.default_parse import read_examples_from_file, convert_examples_to_features

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
examples = read_examples_from_file(".", mode=f"data/{dataset}")
pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index

features = convert_examples_to_features(
    examples,
    label_list=labels,
    max_seq_length=128,
    tokenizer=tokenizer,
    cls_token_at_end=False,
    cls_token=tokenizer.cls_token,
    cls_token_segment_id=0,
    sep_token=tokenizer.sep_token,
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
    pad_token_segment_id=0,
    pad_token_label_id=pad_token_label_id)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

tensordataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)


dataset_test = "eng.testa"
test_examples = read_examples_from_file(".", mode=f"data/{dataset_test}")

test_features = convert_examples_to_features(
    test_examples,
    label_list=labels,
    max_seq_length=128,
    tokenizer=tokenizer,
    cls_token_at_end=False,
    cls_token=tokenizer.cls_token,
    cls_token_segment_id=0,
    sep_token=tokenizer.sep_token,
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
    pad_token_segment_id=0,
    pad_token_label_id=pad_token_label_id)


all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in test_features], dtype=torch.long)

test_tensordataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [85]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 128

# eval_sampler = SequentialSampler(tensordataset)
# eval_dataloader = DataLoader(tensordataset, sampler=eval_sampler, batch_size=BATCH_SIZE)
eval_dataloader = DataLoader(tensordataset, shuffle="True", batch_size=BATCH_SIZE)

# eval_sampler = SequentialSampler(test_tensordataset)
test_eval_dataloader = DataLoader(test_tensordataset, batch_size=BATCH_SIZE)

In [73]:
import torch
from torch import nn


class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding = nn.Embedding(max_length, embedding_dim)
        self.max_length = max_length

    def forward(self, input_ids):

        token_embeddings = self.embedding(input_ids)

        position_ids = torch.arange(self.max_length).unsqueeze(0).repeat(input_ids.size(0), 1).to(input_ids.device)

        position_embeddings = self.position_embedding(position_ids)
        embeddings = token_embeddings + position_embeddings

        return embeddings


class MultiheadSelfAttentionBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 num_heads:int=12,
                 attn_dropout:float=0):
        super().__init__()

        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                    num_heads=num_heads,
                                                    dropout=attn_dropout,
                                                    batch_first=True)

    def forward(self, x, mask):
        x = self.layer_norm(x)
        attn_output, _ = self.multihead_attn(query=x,
                                             key=x,
                                             value=x,
                                             key_padding_mask=mask,
                                             need_weights=False)
        return attn_output


class MLPBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 mlp_size:int=3072,
                 dropout:float=0.1):
        super().__init__()

        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

        self.mlp = nn.Sequential(
            nn.Linear(in_features=embedding_dim,
                      out_features=mlp_size),
            nn.GELU(),
            nn.Dropout(p=dropout),
            nn.Linear(in_features=mlp_size,
                      out_features=embedding_dim),
            nn.Dropout(p=dropout)
        )

    def forward(self, x):
        x = self.layer_norm(x)
        x = self.mlp(x)
        return x


class TransformerEncoderBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 num_heads:int=12,
                 mlp_size:int=3072,
                 mlp_dropout:float=0.1,
                 attn_dropout:float=0):
        super().__init__()

        self.msa_block = MultiheadSelfAttentionBlock(embedding_dim=embedding_dim,
                                                     num_heads=num_heads,
                                                     attn_dropout=attn_dropout)


        self.mlp_block =  MLPBlock(embedding_dim=embedding_dim,
                                   mlp_size=mlp_size,
                                   dropout=mlp_dropout)

    def forward(self, x, mask):
        x =  self.msa_block(x, mask) + x
        x = self.mlp_block(x) + x
        return x


class BERT(nn.Module):
    def __init__(self,
                 vocab_size:int=224,
                 in_channels:int=3,
                 patch_size:int=16,
                 num_transformer_layers:int=12,
                 embedding_dim:int=96, #768
                 mlp_size:int=3072,
                 num_heads:int=12,
                 attn_dropout:float=0,
                 mlp_dropout:float=0.1,
                 embedding_dropout:float=0.1,
                 num_classes:int=9):  #there are 9 tags?
        super().__init__()



        self.class_embedding = nn.Parameter(data=torch.randn(1, 1, embedding_dim),
                                            requires_grad=True)

        self.embedding_dropout = nn.Dropout(p=embedding_dropout)

        self.embedding = Embedding(vocab_size=vocab_size,
                                              embedding_dim=embedding_dim,
                                              max_length=patch_size)

        # self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(embedding_dim=embedding_dim,
        #                                                                     num_heads=num_heads,
        #                                                                     mlp_size=mlp_size,
        #                                                                     mlp_dropout=mlp_dropout) for _ in range(num_transformer_layers)])

        self.transformer_encoder = TransformerEncoderBlock(embedding_dim=embedding_dim,
                                                                            num_heads=num_heads,
                                                                            mlp_size=mlp_size,
                                                                            mlp_dropout=mlp_dropout)

        self.classifier = nn.Sequential(
            nn.LayerNorm(normalized_shape=embedding_dim),
            nn.Linear(in_features=embedding_dim,
                      out_features=num_classes)
        )

    def forward(self, x, mask):
        batch_size = x.shape[0]

        x = self.embedding(x)

        x = self.embedding_dropout(x)
        x = self.transformer_encoder(x, mask)
        x = self.classifier(x)

        return x

In [84]:

model = BERT(vocab_size=len(tokenizer.vocab), embedding_dim=768, patch_size=128)

total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params}')

Total parameters: 29464329


In [89]:
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

pad_token_label_id = torch.nn.CrossEntropyLoss().ignore_index

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    model.train()

    train_loss, train_acc = 0, 0
    for batch_num, batch, in enumerate(dataloader):

        inputs = {"x": batch[0].to(device),
                  # "mask": None}
                  "mask": batch[1].to(device).bool()}

        y_pred = model(**inputs)[0].unsqueeze(dim=0)
        preds = y_pred[0, (batch[3] != pad_token_label_id)[0], :]
        y = (batch[3][0][(batch[3] != pad_token_label_id)[0]]).to(device)

        loss = loss_fn(preds, y)
        # if torch.isnan(loss):
        #   print(inputs)
        #   print(y_pred)
        #   print(preds, y)
        #   break

        if not torch.isnan(loss):
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(preds, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred_class.view(-1))
        # if batch_num % 1000 == 0:
        #     print("train accuracy: "train_acc/(batch_num+1), "batch: ", batch_num)
            # print((y_pred_class == y).sum().item()/len(y_pred_class.view(-1)))
    # print(train_loss)
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch_num, batch in enumerate(dataloader):

            inputs = {"x": batch[0].to(device),
                      "mask": batch[1].bool().to(device)}

            y_pred = model(**inputs)[0].unsqueeze(dim=0)
            preds = y_pred[0, (batch[3] != pad_token_label_id)[0], :]
            y = (batch[3][0][(batch[3] != pad_token_label_id)[0]]).to(device)


            test_pred_labels = preds.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels.view(-1)))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


In [87]:

optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=3e-5,
                             betas=(0.9, 0.999),
                             weight_decay=0.3)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

epochs = 10

# print(train_step(model, eval_dataloader, torch.nn.CrossEntropyLoss(), optimizer, device))
results = train(model, eval_dataloader, test_eval_dataloader, optimizer, torch.nn.CrossEntropyLoss(), epochs, device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1418 | train_acc: 0.7421 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 2 | train_loss: 0.9378 | train_acc: 0.7833 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 3 | train_loss: 0.9969 | train_acc: 0.7462 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 4 | train_loss: 0.8591 | train_acc: 0.7778 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 5 | train_loss: 0.8527 | train_acc: 0.7762 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 6 | train_loss: 0.8779 | train_acc: 0.7699 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 7 | train_loss: 0.9564 | train_acc: 0.7303 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 8 | train_loss: 0.7904 | train_acc: 0.8002 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 9 | train_loss: 0.9179 | train_acc: 0.7478 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 10 | train_loss: 0.8011 | train_acc: 0.7806 | test_loss: 0.0000 | test_acc: 0.8068


In [90]:

optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=3e-5,
                             betas=(0.9, 0.999),
                             weight_decay=0.3)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

epochs = 20

# print(train_step(model, eval_dataloader, torch.nn.CrossEntropyLoss(), optimizer, device))
results = train(model, eval_dataloader, test_eval_dataloader, optimizer, torch.nn.CrossEntropyLoss(), epochs, device)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.0582 | train_acc: 0.7685 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 2 | train_loss: 0.0000 | train_acc: 0.7803 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 3 | train_loss: 0.0000 | train_acc: 0.7938 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 4 | train_loss: 0.0000 | train_acc: 0.7561 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 5 | train_loss: 0.0000 | train_acc: 0.7758 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 6 | train_loss: 0.0000 | train_acc: 0.7813 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 7 | train_loss: 0.0000 | train_acc: 0.7565 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 8 | train_loss: 0.0000 | train_acc: 0.7615 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 9 | train_loss: 0.0000 | train_acc: 0.7598 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 10 | train_loss: 0.0000 | train_acc: 0.7810 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 11 | train_loss: 0.0000 | train_acc: 0.7516 | test_loss: 0.0000 | test_acc: 0.8084
Epoch: 12 | train_l